### Import Libraries

In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE

pd.set_option('display.max_columns', 500)

In [2]:
df_churn = pd.read_csv("../input/telcom-churn/telecom_churn_data.csv")

### Extract high value customers

In [3]:
col = ['total_rech_data_6','av_rech_amt_data_6','total_rech_data_7',
       'total_rech_data_8','av_rech_amt_data_7','av_rech_amt_data_8']

In [4]:
df_churn[col] = df_churn[col].fillna(0)

In [5]:
df_churn['total_rech_data_amt_6'] = (df_churn['total_rech_data_6']*df_churn['av_rech_amt_data_6']) +df_churn['total_rech_amt_6']
df_churn['total_rech_data_amt_7'] = (df_churn['total_rech_data_7']*df_churn['av_rech_amt_data_7']) +df_churn['total_rech_amt_7']
df_churn['total_rech_data_amt_8'] = (df_churn['total_rech_data_8']*df_churn['av_rech_amt_data_8']) +df_churn['total_rech_amt_8']

In [6]:
df_churn['total_avg_amt_6_7'] = (df_churn['total_rech_data_amt_6']+df_churn['total_rech_data_amt_7'])/2

In [7]:
df_churn = df_churn[df_churn['total_avg_amt_6_7']>np.percentile(df_churn['total_avg_amt_6_7'],70)]

In [8]:
df_churn = df_churn.drop('total_avg_amt_6_7', axis=1)

In [9]:
df_churn.shape

### Remove all columns for September month

In [10]:
col = ['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']

In [11]:
df_churn['churned'] = df_churn[col].apply(lambda x:1 if (x[0]+x[1]+x[2]+x[3])==0 else 0, axis=1)

In [12]:
df_churn['churned'].value_counts()

In [13]:
drop_sept_col = df_churn.columns[df_churn.columns.str.contains('_9')]
df_churn = df_churn.drop(drop_sept_col, axis=1)

#### Drop columns with single unique value

In [14]:
value_count_col = pd.DataFrame({'columns':df_churn.columns})

for col in df_churn.columns:
    value_count_col.loc[value_count_col['columns']==col,'unique values'] = df_churn[col].nunique()

In [15]:
col_with_unique_value = value_count_col[value_count_col['unique values']==1]['columns']
df_churn = df_churn.drop(col_with_unique_value, axis=1)
df_churn.shape

#### Drop columns with 40% missing value

In [16]:
miss_per_col = df_churn.isnull().sum()/len(df_churn)
df_churn = df_churn.drop(miss_per_col.index[miss_per_col>.4], axis=1)

#### Convert to correct datatype

In [17]:
cat_var = df_churn.select_dtypes(include='object')
cat_var.head()

In [18]:
for col in cat_var:
    df_churn[col] = pd.to_datetime(df_churn[col])

In [19]:
df_churn[cat_var.columns] = df_churn[cat_var.columns].fillna(0)

In [20]:
df_churn['date_of_last_rech_6'] = df_churn['date_of_last_rech_6'].apply(lambda x: x.day if x!=0 else x)
df_churn['date_of_last_rech_7'] = df_churn['date_of_last_rech_7'].apply(lambda x: x.day if x!=0 else x)
df_churn['date_of_last_rech_8'] = df_churn['date_of_last_rech_8'].apply(lambda x: x.day if x!=0 else x)

### Data Visualisation

In [21]:
# create box plot for  6th, 7th and 8th month
def plot_box_chart(attribute):
    plt.figure(figsize=(15,10))
    plt.subplot(2,3,1)
    sns.boxplot(data=df_churn, y=attribute+"_6",x="churned",hue="churned",
                showfliers=False,palette=("plasma"))
    plt.subplot(2,3,2)
    sns.boxplot(data=df_churn, y=attribute+"_7",x="churned",hue="churned",
                showfliers=False,palette=("plasma"))
    plt.subplot(2,3,3)
    sns.boxplot(data=df_churn, y=attribute+"_8",x="churned",hue="churned",
                showfliers=False,palette=("plasma"))
    plt.show()

In [22]:
plot_box_chart('date_of_last_rech')

In [23]:
# Ploting for total recharge amount:
plot_box_chart('total_rech_amt')

In [24]:
# Ploting for total recharge amount for data:
plot_box_chart('total_rech_data_amt')

In [25]:
# Ploting for maximum recharge amount for data:
plot_box_chart('max_rech_amt')

In [26]:
# Ploting for Total recharge for Number:
plot_box_chart('total_rech_num')

In [27]:
# Ploting for last day recharge amount:
plot_box_chart('last_day_rch_amt')

In [28]:
usage_2g_and_3g = df_churn.columns[df_churn.columns.str.contains('2g|3g',regex=True)]
usage_2g_and_3g

In [29]:
df_churn = df_churn.drop('sep_vbc_3g', axis=1)

In [30]:
# Ploting for volume of 2G and 3G usage columns:
plot_box_chart('vol_2g_mb')

In [31]:
# Ploting for volume of 2G and 3G usage columns:
plot_box_chart('vol_3g_mb')

In [32]:
def plot_mean_bar_chart(df,columns_list):
    df_0 = df[df.churned==0].filter(columns_list)
    df_1 = df[df.churned==1].filter(columns_list)
    
    mean_df_0 = pd.DataFrame([df_0.mean()],index={'Non Churn'})
    mean_df_1 = pd.DataFrame([df_1.mean()],index={'Churn'})
    
    frames = [mean_df_0, mean_df_1]
    mean_bar = pd.concat(frames)

    mean_bar.T.plot.bar(figsize=(10,4),rot=0)
    plt.show()

In [33]:
col = ['monthly_2g_6', 'monthly_2g_7', 'monthly_2g_8', 'monthly_3g_6', 'monthly_3g_7', 'monthly_3g_8']

In [34]:
plot_mean_bar_chart(df_churn,col)

In [35]:
col = ['jun_vbc_3g', 'jul_vbc_3g', 'aug_vbc_3g']
plot_mean_bar_chart(df_churn,col)

In [36]:
col = ['sachet_2g_6', 'sachet_2g_7', 'sachet_2g_8', 'sachet_3g_6', 'sachet_3g_7', 'sachet_3g_8']
plot_mean_bar_chart(df_churn,col)

In [37]:
plot_box_chart('arpu')

In [38]:
mou_cols = df_churn.columns[df_churn.columns.str.contains('mou')]
mou_cols

In [39]:
df_churn[mou_cols] = df_churn[mou_cols].fillna(0)

In [40]:
mou_cols_6 = df_churn.columns[df_churn.columns.str.contains('mou_6')]
mou_cols_6

In [41]:
df_churn[mou_cols_6].head()

#### Drop highly correlated columns

In [42]:
corr_col = df_churn.columns[df_churn.columns.str.contains('total_og_mou|std_og_mou|loc_og_mou|total_ic_mou|std_ic_mou|loc_ic_mou',regex=True)]
df_churn = df_churn.drop(corr_col, axis=1)

In [43]:
plot_box_chart('offnet_mou')

In [44]:
plot_box_chart('onnet_mou')

In [45]:
# tn_range = [0, 6, 12, 24, 60, 61]
# tn_label = [ '0-6 Months', '6-12 Months', '1-2 Yrs', '2-5 Yrs', '5 Yrs and above']
# tenure_data['tenure_range'] = pd.cut(tenure_data['tenure_in_months'], tn_range, labels=tn_label)

In [46]:
df_churn = df_churn.fillna(0)

#### Derive new columns

In [47]:
col_list = df_churn.filter(regex='_6|_7').columns.str[:-2]
for col in set(col_list):
    avg_col_name = col+"_avg67"
    col_6 = col+"_6"
    col_7 = col+"_7"
    df_churn[avg_col_name] = (df_churn[col_6]  + df_churn[col_7])/ 2
    print(avg_col_name)

In [48]:
col_list_to_drop = df_churn.filter(regex='_6|_7')
df_churn.drop(col_list_to_drop,axis=1,inplace=True)
df_churn.shape

In [49]:
y = df_churn.pop('churned')
X = df_churn

#### Using SMOTE to tackle Class imbalance

In [50]:
# ros = RandomOverSampler(random_state=42)
smote = SMOTE()
# fit predictor and target variable
X_ros, y_ros = smote.fit_resample(X, y)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.2, random_state=42)

In [52]:
col = X_train.columns

#### Data Scaling

In [53]:
sc = StandardScaler()
X_train[col] = sc.fit_transform(X_train[col])
X_test[col] = sc.transform(X_test[col])

#### Principle Component Analysis

In [54]:
pca = PCA(n_components=60, random_state=42)

In [55]:
X_pca_train = pca.fit_transform(X_train)
X_pca_test = pca.transform(X_test)

In [56]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(1,len(var_cumu)+1), var_cumu)

### Data Modeling

In [57]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [58]:
# Create the parameter grid based on the results of random search 
params = {
    'n_estimators': [200],
    'max_depth': [15, 20],
    'min_samples_leaf': [10,20],
    'max_features': [30,40],
    'min_samples_split': [10,20]
}

In [59]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=classifier_rf, param_grid=params, 
                          cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [60]:
%%time
grid_search.fit(X_pca_train,y_train)

In [61]:
grid_search.best_estimator_

In [ ]:
classifier_rf = RandomForestClassifier(max_depth=20, max_features=40, min_samples_leaf=10,
                       min_samples_split=10, n_estimators=200, n_jobs=-1,random_state=42, oob_score=True)

In [ ]:
classifier_rf.fit(X_pca_train, y_train)

In [92]:
def print_model_metrics(y_test,y_pred,model_name):
#     header(model_name+" Model Stats Scores Summary : ")
    cp = confusion_matrix(y_test,y_pred)
#     plt.figure()
#     plot_confusion_matrix(cp)
#     plt.show()
    
    accuracy = round(accuracy_score(y_test,y_pred),2)
    recall = round(recall_score(y_test,y_pred),2)
    precision = round(precision_score(y_test,y_pred),2)
    auc = round(roc_auc_score(y_test,y_pred),2)
    f1 = round(f1_score(y_test,y_pred),2)
    
    data = [[model_name,accuracy,recall,precision,auc,f1]] 
    df = pd.DataFrame(data, columns = ['Model', 'Accuracy','Precision','Recall','AUC','F1'])
#     add_to_global_summary(df)
    return df 

In [ ]:
print_model_metrics(y_test, classifier_rf.predict(X_pca_test),"Ramdom Forest")

In [ ]:
print(classification_report(y_test, classifier_rf.predict(X_test)))

In [ ]:
def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(y_train, dt_classifier.predict(X_train)))
    print("Precision :", precision_score(y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(y_test, dt_classifier.predict(X_test)))
    print("Precision :", precision_score(y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))

In [ ]:
evaluate_model(classifier_rf)

In [ ]:
clf_xg = XGBClassifier(n_estimators=200, learning_rate=0.1, random_state=42, n_jobs=-1)
clf_xg.fit(X_train,y_train)

In [ ]:
print(classification_report(y_test, clf_xg.predict(X_test)))

In [ ]:
evaluate_model(clf_xg)

### Identify important predcitor variables

#### Logistic Regression with RFE

In [62]:
lr = LogisticRegression(solver='liblinear')

In [63]:
rfe = RFE(lr, 15)             # running RFE with 13 variables as output
rfe = rfe.fit(X_train, y_train)

In [64]:
col = X_train.columns[rfe.support_]

In [65]:
X_train.columns[~rfe.support_]

In [66]:
y_train_pred_final = pd.DataFrame({'CustID':y_train.index, 'Churn':y_train.values})

In [67]:
def VIF(X_train_new):
    vif = pd.DataFrame()
    X = X_train_new
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

def stats_model(X_train_new):
    X_train_sm = sm.add_constant(X_train_new)
    lm = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial()).fit()   # Running the linear model
    y_train_pred = lm.predict(X_train_sm).values.reshape(-1)
    y_train_pred_final['Churn_Prob'] = y_train_pred
    y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)
    return lm, y_train_pred_final

In [68]:
res, y_train_pred_final = stats_model(X_train[col])
res.summary()

In [69]:
y_train_pred_final.head()

In [70]:
vif = VIF(X_train[col])
vif

In [71]:
print(accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

In [72]:
col = col.drop('total_rech_data_8',1)

In [73]:
res, y_train_pred_final = stats_model(X_train[col])
res.summary()

In [74]:
vif = VIF(X_train[col])
vif

In [75]:
print(accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

In [76]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [77]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [78]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci','precision','recall'])
from sklearn.metrics import confusion_matrix

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = confusion_matrix(y_train_pred_final.Churn, y_train_pred_final[i] )
    pre_score = precision_score(y_train_pred_final.Churn, y_train_pred_final[i] )
    recall = recall_score(y_train_pred_final.Churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci, pre_score,recall]
print(cutoff_df)

In [79]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

In [80]:
from sklearn.metrics import precision_recall_curve

In [81]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

In [82]:
plt.plot(thresholds, p[:-1], "g-")
plt.plot(thresholds, r[:-1], "r-")
plt.show()

#### Cut-off should be 0.6

In [84]:
X_test = X_test[col]
X_test_sm = sm.add_constant(X_test)
y_test_pred = res.predict(X_test_sm)

In [85]:
# Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)
# Converting y_test to dataframe
y_test_df = pd.DataFrame(y_test)
# Putting CustID to index
y_test_df['CustID'] = y_test_df.index
# Removing index for both dataframes to append them side by side 
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)
# Appending y_test_df and y_pred_1
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)
y_pred_final.head()

In [86]:
# Renaming the column 
y_pred_final= y_pred_final.rename(columns={ 0 : 'Churn_Prob'})
#Mapping the test data
y_pred_final['final_predicted'] = y_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.6 else 0)
y_pred_final.head()

In [87]:
accuracy_score(y_pred_final.churned, y_pred_final.final_predicted)

In [88]:
precision_score(y_pred_final.churned, y_pred_final.final_predicted)

In [89]:
classifier_rf = RandomForestClassifier(random_state=42, oob_score=True)

In [90]:
classifier_rf.fit(X_train[col],y_train)

In [93]:
print_model_metrics(y_test, classifier_rf.predict(X_test[col]),"Random Forest")

#### Since we are focusing on Precision, we will take it as metrics

In [94]:
imp_df = pd.DataFrame({
    "Varname": X_train[col].columns,
    "Imp": classifier_rf.feature_importances_
})

In [95]:
imp_df.sort_values(by="Imp", ascending=False)[:10]

#### Below are the important predictor variables:
Total recharge data,
Last recharge date,
Last recharge amount,
Incoming call minute of usage
#### We should monitor the above variables and if there is any descrease in the values, we should provide offer to those users